In [9]:
#!pip install gurobipy
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 15.2 MB/s eta 0:00:00


In [10]:
import pandas as pd
import numpy as np
#import gurobipy as gp
#from gurobipy import GRB
from mip import *

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
data = pd.read_csv('/content/drive/MyDrive/종합설계/small_data.csv') ########################디렉토리 경로 바꾸기

In [13]:
data['Delay'].sum()

1.4

In [14]:
data

,선박명,ATB,ATD,Total,ETB,ETD,배길이(m),Delay
0,MING YUE,1.4,11.0,187,0.0,11.0,81,0.0
1,REVERENCE,3.5,17.0,966,3.0,17.0,142,0.0
2,YA LU JIANG,15.0,19.5,102,15.0,19.0,110,0.5
3,SUNNY CAMELLIA,16.5,28.0,632,16.0,28.0,141,0.0
4,WAN HAI 317,25.5,41.0,837,25.0,41.0,212,0.0
5,HANSUNG INCHEON,29.0,38.0,349,29.0,38.0,137,0.0
6,SKY RAINBOW,30.0,41.9,607,30.0,41.0,172,0.9


In [15]:
mip = Model('BACAP', MINIMIZE, solver_name='CBC')

In [16]:
#Paramaters
N = 7       #total vessel number
gap_y = 20  #distance interval(unit = meter)
gap_x = 1   #time interval(unit = minute)
T = 42      #planning period
L = 800     #quay length
Q = 7       #total QC numbers
qmin = 1    #minimum QC/hr for vessel i
qmax = 4    #maximum QC/hr for vessel i
p = 25      #productivity

M = 1e5        #big M

In [17]:
import math

In [18]:
r = np.array(data['Total'])     #total container numbers
cost = 15*r                   #penalty cost
m = r/p                         #total number of QC required for vessel i
ETA = np.array(data['ETB']).astype(int)     #ETA for vessel i
ETD = np.array(data['ETD'])     #ETD for vessel i
l = np.array(data['배길이(m)']) #vessel length


r1 = r
for i in range(N):
  m[i] = math.ceil(m[i])

In [19]:
b = [mip.add_var(name = 'berthing time',var_type = CONTINUOUS) for i in range(N)]
x = [mip.add_var(name = 'berthing position',var_type = CONTINUOUS) for i in range(N)]
d = [mip.add_var(name = 'departure time',var_type = CONTINUOUS, ) for i in range(N)]
temp = [mip.add_var(name = 'objective_temp',var_type = CONTINUOUS, ) for i in range(N)]
#z = [mip.add_var(name='z_{}'.format(i), var_type=BINARY) for i in range(N)]
#y = [mip.add_var(name = 'delay',var_type = CONTINUOUS, ) for i in range(N)]
#x = mip.addVars(16, vtype = GRB.CONTINUOUS, name = 'berthing position')
#d = mip.addVars(16, vtype = GRB.CONTINUOUS, name = 'departure time')
#d = np.full(16, 6480.0) #planning period 의 끝값으로 초기화

In [20]:
vx = {(i, j): mip.add_var(var_type=BINARY, name=f"vessel {i} is left of vessel {j}")for i in range(N) for j in range(N)  if i != j}
vy = {(i, j): mip.add_var(var_type=BINARY, name=f"vessel {i} is below vessel {j}")for i in range(N) for j in range(N) if i != j}


In [21]:
print(vx[(4,1)]-1) #####

+ vessel 4 is left of vessel 1 - 1


In [22]:
q = {(i, t, n): mip.add_var(name = 'qitni', var_type = BINARY) for i in range(N) for t in range(T + 1) for n in range(qmin, qmax + 1)}

In [23]:
#qitn_i

In [24]:
#set objectives
# temp1 = xsum((cost[i]* d[i]) for i in range(N))
# temp2 = xsum((cost[i]* ETD[i]) for i in range(N))
mip.objective = minimize(xsum((cost[i]* temp[i]) for i in range(N)))


In [25]:
print(mip.objective)

+ 2805.0objective_temp + 14490.0objective_temp + 1530.0objective_temp + 9480.0objective_temp + 12555.0objective_temp + 5235.0objective_temp + 9105.0objective_temp 


# constraint

In [26]:
V1 = []
V2 = [0,1,2,3,4,5,6]
b_1 = []
x_1 = []
# [20, 600, 20, 150, 300, 0, 160]
tau = 0

In [27]:
data

,선박명,ATB,ATD,Total,ETB,ETD,배길이(m),Delay
0,MING YUE,1.4,11.0,187,0.0,11.0,81,0.0
1,REVERENCE,3.5,17.0,966,3.0,17.0,142,0.0
2,YA LU JIANG,15.0,19.5,102,15.0,19.0,110,0.5
3,SUNNY CAMELLIA,16.5,28.0,632,16.0,28.0,141,0.0
4,WAN HAI 317,25.5,41.0,837,25.0,41.0,212,0.0
5,HANSUNG INCHEON,29.0,38.0,349,29.0,38.0,137,0.0
6,SKY RAINBOW,30.0,41.9,607,30.0,41.0,172,0.9


In [28]:
for i in V1:
  mip += b[i] == b_1[i]  #2
  mip += x[i] == x_1[i]  #3

In [29]:
for i in V2:
  mip += (b[i] >= ETA[i])  #4
  mip += (x[i] <= L-l[i])  #5

In [30]:
for i in range(N):
  for j in range(N):
    if i != j:
      mip += b[j] >= d[i] + gap_x + M*(vx[(i,j)]-1)  #6
      mip += x[j] >= x[i] + l[i] + gap_y + M*(vy[(i,j)]-1)  #7
      mip += vx[(i,j)] + vx[(j,i)] + vy[(i,j)] + vy[(j,i)] >= 1  #8

In [31]:
for i in V2:
  for t in range(max(ETA[i], tau), T):
    for n in range(qmin, qmax+1):
      mip += b[i] <= t*q[(i,t,n)] + M*(1-q[(i,t,n)])  #9
      
for i in V1:
  for t in range(max(ETA[i], tau), T):
    for n in range(qmin, qmax+1):
      mip += d[i] >= t*q[(i,t,n)] + 1  #10
      
for i in range(N):
  mip += d[i] - b[i] <= math.ceil(m[i]/qmin)  #11
  
for i in V1:
  mip += xsum(n*q[(i,t,n)] for t in range(tau,T) for n in range(qmin, qmax+1)) >= math.ceil(r1[i]/p)  #12 
  
for i in V2:
  mip += xsum(n*q[(i,t,n)] for t in range(tau,T) for n in range(qmin, qmax+1)) >= m[i]  #13
  
for t in range(tau, T):
  mip += xsum(n*q[(i,t,n)] for i in range(N) for n in range(qmin,qmax+1)) <= Q  #14
  
for i in range(N):
  for t in range(tau,T):
    mip += xsum(q[(i,t,n)] for n in range(qmin, qmax+1)) <= 1   #15

for i in range(N):
  for j in range(N):
    if i != j:
      mip += b[i] - b[j] + M * vx[(i, j)] >= 0 ## toggle
      mip += b[i] - b[j] - M * (1 - vx[(i, j)]) <= -1 ## toggle
      mip += x[i] - x[j] + M * vy[(i, j)] >= 0 ## toggle
      mip += x[i] - x[j] - M * (1 - vy[(i, j)]) <= -1 ## toggle


In [32]:
for i in range(N):
    mip += temp[i] == d[i]-ETD[i]
    

In [33]:
mip.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [34]:
mip.objective_value

0.0

In [35]:
data

,선박명,ATB,ATD,Total,ETB,ETD,배길이(m),Delay
0,MING YUE,1.4,11.0,187,0.0,11.0,81,0.0
1,REVERENCE,3.5,17.0,966,3.0,17.0,142,0.0
2,YA LU JIANG,15.0,19.5,102,15.0,19.0,110,0.5
3,SUNNY CAMELLIA,16.5,28.0,632,16.0,28.0,141,0.0
4,WAN HAI 317,25.5,41.0,837,25.0,41.0,212,0.0
5,HANSUNG INCHEON,29.0,38.0,349,29.0,38.0,137,0.0
6,SKY RAINBOW,30.0,41.9,607,30.0,41.0,172,0.9


In [36]:
for i in range(N):
  print('departure time[',i,']=',d[i].x)
  print('berthing position[',i,']=',x[i].x)
  print('berthing time[',i,']=',b[i].x)

  print()

departure time[ 0 ]= 11.0
berthing position[ 0 ]= 0.0
berthing time[ 0 ]= 8.0

departure time[ 1 ]= 17.0
berthing position[ 1 ]= 0.0
berthing time[ 1 ]= 12.0

departure time[ 2 ]= 19.0
berthing position[ 2 ]= 0.0
berthing time[ 2 ]= 18.0

departure time[ 3 ]= 28.0
berthing position[ 3 ]= 0.0
berthing time[ 3 ]= 28.0

departure time[ 4 ]= 41.0
berthing position[ 4 ]= 0.0
berthing time[ 4 ]= 34.0

departure time[ 5 ]= 38.0
berthing position[ 5 ]= 0.0
berthing time[ 5 ]= 100000.0

departure time[ 6 ]= 41.0
berthing position[ 6 ]= 0.0
berthing time[ 6 ]= 42.0



In [ ]:
for i in range(N):
  for j in range(N):
    if i !=j :
      print(vx[(i,j)].x)
      print(vy[(i,j)].x)
      print()
      

0.0
0.0

0.0
0.0

1.0
0.0

0.0
0.0

1.0
0.0

1.0
0.0

0.0
1.0

1.0
0.0

0.0
1.0

0.0
0.0

1.0
0.0

0.0
1.0

0.0
1.0

0.0
0.0

0.0
0.0

0.0
0.0

0.0
0.0

1.0
0.0

0.0
1.0

0.0
0.0

0.0
1.0

1.0
0.0

1.0
0.0

1.0
0.0

0.0
1.0

0.0
1.0

0.0
1.0

0.0
1.0

0.0
1.0

0.0
1.0

0.0
1.0

0.0
0.0

0.0
1.0

0.0
1.0

0.0
0.0

0.0
1.0

0.0
0.0

0.0
0.0

0.0
0.0

0.0
0.0

0.0
0.0

1.0
0.0

